
# Report: Correlation between rates of Gold Price and Crude Oil Price
=====================================================================================================
# Author: **[Parth Ghinaiya](https://github.com/parth-ghinaiya)**
-----------------------------------------------------------------------------------------------------
## Preparation
### Install Dependencies

In [5]:
%pip install --upgrade pip
%pip install matplotlib==3.8.2
%pip install numpy==1.26.3
%pip install pandas==1.5.3
%pip install ==1.11.4
%pip install SQLAlchemy==1.4.46
%pip install kaggle==1.5.16

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
zsh:1: =1.11.4 not found
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import Dependencies

In [2]:
import os
import subprocess
import urllib.request
import zipfile
# import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# from scipy.stats import pearsonr
from sqlalchemy import create_engine

### Load Data into Sqlite Database

In [6]:
# Run datapipeline to create database if it doesn't exist
subprocess.run(["./pipeline.sh"])

CompletedProcess(args=['./pipeline.sh'], returncode=0)

### Retrieve Data from Database

In [9]:
# Load data from database
crude_oil_price_db_engine = create_engine('sqlite:///../data/crude_oil_price.sqlite')
gold_price_db_engine = create_engine('sqlite:///../data/gold_price.sqlite')

# Create dataframe from database
crude_oil_price_df = pd.read_sql_table('crude_oil_price', crude_oil_price_db_engine)
gold_price_df = pd.read_sql_table('gold_prices', gold_price_db_engine)

          date    price  percentChange  change
0   1983-03-30  29.2700            NaN     NaN
1   1983-04-04  30.6300          4.646  1.3600
2   1983-05-02  30.2500         -1.241 -0.3800
3   1983-06-01  31.3800          3.736  1.1300
4   1983-07-01  32.0000          1.976  0.6200
..         ...      ...            ...     ...
486 2023-09-01  88.8000          7.040  5.8400
487 2023-10-02  80.5000         -9.347 -8.3000
488 2023-11-01  76.0500         -5.528 -4.4500
489 2023-12-01  71.6500         -5.786 -4.4000
490 2024-01-02  73.1716          2.124  1.5216

[491 rows x 4 columns]


### Analyze Data